# Model Setup Script with Clouds 

You should already understand the basics of a `model_setup_cldfree.py` file. Here we will show how to add a case with a cloud retrieval, and identify the largest differences between a model setup with and without clouds.

# Sets and Setups for Ultranest

## Param Set

You can see the biggest difference is that there is now a new line of code where we add the parameters associated with a typical `virga` run, that being `logkzz` and `fsed` (note that `offset_hst` and `offset_jwst` is data specific, you probably won't need it in your run).

In [1]:
class param_set:
    """
    This is purely for book keeping what parameters you have run in each retrieval.
    It helps if you keep variables uniform.
    THINGS TO CHECK:
    1) Make sure that the variables here match how you are unpacking your cube in the model_set class and prior_set
    2) Make sure that the variable names here match the function names in model_set and prior_set
    """
    cld_free = ','.join(grid_parameters_unique.keys())+',offset_hst,offset_jwst'
    cld_free_virga = ','.join(grid_parameters_unique.keys())+',offset_hst,offset_jwst,logkzz,fsed'

NameError: name 'grid_parameters_unique' is not defined

## Guesses Set

There is now a line of code to obtain `virga` parameters.

In [ ]:
class guesses_set: 
    """
    Optional! 
    This is only used if you want to test your initial functions before running 
    the full retrievals. See script test.py
    """
    #completely random guesses just to make sure it runs
    cld_free=[grid_parameters_unique[i][0]
             for i in grid_parameters_unique.keys()] + [-.005,-.005]
    cld_free_virga=[grid_parameters_unique[i][0]
             for i in grid_parameters_unique.keys()] + [-.005,-.005,10,1]

## Model Set

A whole other defintion has been created for specifically `virga`, where we first get our "core" parameters, or the parameters that are not related to `virga`. Next, we grab the P-T profile and chemical equilbirum from the non-virga run, and then add extra parameters for the clouds (`kzz`, `fsed`, etc). Finally, we recompute the spectrum (not climate!) in order to get the full model.

In [ ]:
class model_set:
    """100e-6=1e-4
    This is your full model set. It will include all the functions you want to test
    for a particular data set.
    """     
    def cld_free(cube): 
        final_goal = cube[0:len(grid_parameters_unique.keys())]
        spectra_interp = lyz.custom_interp(final_goal, fitter, grid_name)
        micron = fitter.wavelength[grid_name]
        wno = 1e4/fitter.wavelength[grid_name] 
        return wno, spectra_interp

    def cld_free_virga(cube,planet=planet_og):
        #1) get core parametrs
        final_goal = cube[0:len(grid_parameters_unique.keys())]
        off_hst = cube[-4]
        off_jwst = cube[-3]     
        kzz = 10**cube[-2]
        fsed = 10**cube[-1]  
        
        #2) from core parameters get PT profile and chemistry
        dd,indz = tree.query(final_goal,k=1)
        near = grid_points.iloc[indz]
        file = os.path.join(location,
            f"wasp-17btint{near['tint']}-rfacv{near['heat_redis']}-mh{near['mh']}-cto{near['cto']}.nc")
        
        ds = jdi.xr.load_dataset(file)
        df = {'pressure':ds.coords['pressure'].values}
        for i in [i for i in ds.data_vars.keys() if 'transit' not in i]:
            if i not in ['opd','ssa','asy']:
                #only get single coord pressure stuff
                if (len(ds.data_vars[i].values.shape)==1 &
                            ('pressure' in ds.data_vars[i].coords)):
                    df[i]=ds.data_vars[i].values        
        
        #mh = 10**float(near['mh'])
        
        for i in opacity.keys(): planet[i].atmosphere(df=jdi.pd.DataFrame(df))    
        
        #3) bonus parameters: add clouds 
        for i in opacity.keys(): planet[i].inputs['atmosphere']['profile']['kz'] = kzz
        for i in opacity.keys(): 
            planet[i].virga(['SiO2','Al2O3'], '/data/virga_dbs/virga_0,3_15_R300/', 
                 fsed=fsed, verbose=False,sig=1.2,
                                                mh=1)#, gas_mmr={'SiO2':1e-5}) full_output=True only exists for virga_3d
        
        x = []
        y = []
        
        #3) have to recompute the "spectrum" (not the climate!)
        for i in opacity.keys(): 
            out = planet[i].spectrum(opacity[i], calculation='transmission',full_output=True)
            x += list(out['wavenumber'])
            y += list(out['transit_depth'])
        
        combined = sorted(zip(x, y), key=lambda pair: pair[0])

        sorted_x = np.array([pair[0] for pair in combined])
        sorted_y = np.array([pair[1] for pair in combined])
        #adjust less than 2 micron for HST
        sorted_y[1e4/sorted_x<2] = sorted_y[1e4/sorted_x<2]+off_hst
        #adjust greater than 2 micron for JWST
        sorted_y[1e4/sorted_x>=2] = sorted_y[1e4/sorted_x>=2]+off_jwst                
        return sorted_x, sorted_y

## Prior Set

Again, a new defintion for virga, where we shift JWST and HST offsets (specific only for our dataset, you shouldn't need it). After, we grab the parameters for `virga`, stored in the prior for use.

In [ ]:
class prior_set:
    """
    Store all your priors. You should have the same exact function names in here as
    you do in model_set and param_set

    Make sure the order of the unpacked cube follows the unpacking in your model 
    set and in your parameter set. 
    #pymultinest: http://johannesbuchner.github.io/pymultinest-tutorial/example1.html
    """   
    def cld_free(cube):#,ndim, nparams):
        params = cube.copy()
        for i,key in enumerate(grid_parameters_unique): 
            minn = np.min(grid_parameters_unique[key]) 
            maxx = np.max(grid_parameters_unique[key]) 
            params[i] = minn + (maxx-minn)*params[i]
        return params


    def cld_free_virga(cube):#,ndim, nparams):
        params = cube.copy()
        for i,key in enumerate(grid_parameters_unique): 
            minn = np.min(grid_parameters_unique[key]) 
            maxx = np.max(grid_parameters_unique[key]) 
            params[i] = minn + (maxx-minn)*params[i]
        
        #shift hst
        i+=1;params[i] = -0.01*params[i]
        #shift jwst
        i+=1;params[i] = -0.01*params[i]
        
        #logkzz
        i+=1;params[i] = 7 + 4*params[i]
        #log fsed 
        i+=1;params[i] = -1 + 2*params[i]
        return params